In [1]:
from ibm_watson import AssistantV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import json
import requests
import webbrowser
from threading import Thread
import struct
import time
import hashlib
import base64
import socket
import time
import types
import multiprocessing
import os
import collections
import pandas as pd
current_path = os.path.abspath(os.getcwd())

In [2]:
# Import another ipynb file into this file
%run Recommendation/PlaceSearch.ipynb
mode = "initialize"
preference = {}
using_watson = True
# IBM Watson Authenication
authenticator = IAMAuthenticator('qUj4Zg8prt9j2jOCIBamphxWFyYXkKS8QmYZk1E-n-mM')
assistant = AssistantV2(
    version='2021-11-11',
    authenticator=authenticator
)
assistant.set_service_url('https://api.us-south.assistant.watson.cloud.ibm.com')
sessionID = assistant.create_session(
    assistant_id='46543a05-d01d-4da7-a729-6daa8cb1e5b4',
).get_result()
valid_hotel_zipcode = ["10001", "10025"]
class socketProgramming(Thread):
    def __init__(self, connection):
        Thread.__init__(self)
        self.con = connection
        self.connected = False
    def run(self):
        global mode
        global preference
        global using_watson
        global authenticator
        global assistant
        global sessionID
        global valid_hotel_zipcode
        while True:
            if not self.connected:
                header = self.request()
                secKey = header['Sec-WebSocket-Key']
                acceptKey = self.generateKey(secKey)
                response = "HTTP/1.1 101 Switching Protocols\r\n"
                response += "Upgrade: websocket\r\n"
                response += "Connection: Upgrade\r\n"
                response += "Sec-WebSocket-Accept: %s\r\n\r\n" % (acceptKey.decode('utf-8'))
                self.con.send(response.encode())
                self.connected = True
                if(mode=="initialize"):
                    mode = "get_order"
            elif mode == "get_order":
                opcode = self.getOpcode()
                if opcode == 8:
                    self.con.close()
                self.getDataLength()
                clientData = self.readData()
                if str(clientData) == 'restart' or str(clientData) == 'Restart':
                    # Initialize a new IBM Watson Assistant instance.
                    authenticator = IAMAuthenticator('qUj4Zg8prt9j2jOCIBamphxWFyYXkKS8QmYZk1E-n-mM')
                    assistant = AssistantV2(
                        version='2021-11-11',
                        authenticator=authenticator
                    )
                    assistant.set_service_url('https://api.us-south.assistant.watson.cloud.ibm.com')
                    sessionID = assistant.create_session(
                        assistant_id='46543a05-d01d-4da7-a729-6daa8cb1e5b4',
                    ).get_result()
                    using_watson = True
                # If using_watson, will send users' inputs to IBM Watson assistant and generate responses.
                if using_watson:
                    ans, choices, preference = self.answer(clientData)
                    if choices:
                        ans += '\n' + " or ".join(choices)
                    if "OK. I will start" in ans:
                        using_watson = False
                        print(preference)
                    self.sendData(ans)
                    
                # If not using_watson, will send extracted features to PlaceSearch.ipynb and generate recommendations.
                if not using_watson:
                    can_find_hotel = True
                    self.sendData("Wating....... Please wait for a moment. Thanks for your patience!")
                    print(preference)
                    items = [preference["place"], "play"]
                    place = preference["place"]
                    requirement = {
                        preference["place"] : {
                            'price' : preference['price'] if 'price' in preference else 'no_require',
                            'rating' : preference['rating'] if 'rating' in preference else 'no_require',
                            "safety":"high"
                        }, 
                        "play": {"rating":"high", "safety":"high"}
                    }
                    
                    if place == 'hotel':
                        if preference['zipcode'] not in valid_hotel_zipcode:
                            can_find_hotel = False
                        requirement['hotel']['bed'] = preference['bed'][1:-1] if 'bed' in preference else None
                    if not can_find_hotel:
                        del requirement['hotel']
                        items.pop(0)
                    
                    print(requirement)
                    output = recommendPlace(
                                    requirement,
                                    items, 
                                    preference['zipcode'], 
                                    5000,
                                    "2021-12-19",
                                    3
                                   )

                    ans = "Here is what we found \n"
                    ans += ' \n'
                    
                    if not can_find_hotel:
                        ans += "Sorry, we have already run out of free-trials for Hotel API, you could test finding hotels using zip codes 10000 and 10025"
                    else:
                        for index, row in output[preference["place"]].iterrows():
                            address = row['formatted_address']
                            name = row['name']
                            ans += address + ', ' + name + '\n'
                            ans += '\n'
                        
                    ans += "\n We also found some places interesting for you! \n"
                    for index, row in output['play'].iterrows():
                        address = row['formatted_address']
                        name = row['name']
                        ans += address + ', ' + name + '\n'
                        ans += "\n"


                    ans += "\n Here are recommendations for places for fun by analyzing Tweets online! \n"
    
                    for index, row in output['twitter'].iterrows():
                        name = row['acutal_name']
                        ans += name + '\n'
                        ans += '\n'
                    self.sendData(ans)
                    preference = {}
                    
    def request(self):
        request = self.con.recv(1024).decode()
        request_list = request.split('\r\n')
        header = {}
        for n in request_list:
            if ': ' in n:
                temp = n.split(': ')
                header[temp[0]] = temp[1]
        return header
    def generateKey(self, secKey):
        sh_key = hashlib.sha1()
        # Generate a magic key with number common used string 258EAFA5-E914-47DA-95CA-C5AB0DC85B11
        sh_key.update((secKey + '258EAFA5-E914-47DA-95CA-C5AB0DC85B11').encode())
        sh_key_result = sh_key.digest()
        acceptKey = base64.b64encode(sh_key_result)
        return acceptKey
    def getOpcode(self):
        eight_bits = self.con.recv(1)
        eight_bits = struct.unpack('B', eight_bits)[0]
        opcode = eight_bits & 0b00001111
        return opcode
    def getDataLength(self):
        data = self.con.recv(1)
        data = struct.unpack('B', data)[0]
        masking = data >> 7
        dataLength = data & 0b01111111
        if dataLength <= 125:
            data_length = dataLength
        elif dataLength == 126:
            data_length = struct.unpack('H', self.con.recv(2))[0]
        elif dataLength == 127:
            data_length = struct.unpack('Q', self.con.recv(8))[0]
        self.masking = masking
        self.data_length = data_length
    def readData(self):
        if self.masking == 1:
            maskingKey = self.con.recv(4)
        data = self.con.recv(self.data_length)
        if self.masking == 1:
            i = 0
            trueData = ''
            for d in data:
                trueData += chr(d ^ maskingKey[i % 4])
                i += 1
            return trueData
        else:
            return data
    def sendData(self, text):
        sendData = ''
        sendData = struct.pack('!B', 0x81)
        length = len(text)
        if length <= 125:
            sendData += struct.pack('!B', length)
        elif length <= 65536:
            sendData += struct.pack('!B', 126)
            sendData += struct.pack('!H', length)
        elif length == 127:
            sendData += struct.pack('!B', 127)
            sendData += struct.pack('!Q', length)
        sendData += struct.pack('!%ds' % (length), text.encode())
        self.con.sendall(sendData)

    def answer(self,data):
        entity_choices = ['restaurant', 'hotel']
        feature_choices = ['rating', 'Price', 'Room_type']
        return reply(data, entity_choices, feature_choices, preference)
    
# Function to get responses from IBM Watson Assistant
def reply(input_data, entity_choices, feature_choices, preference):
    response = assistant.message(
    assistant_id='46543a05-d01d-4da7-a729-6daa8cb1e5b4',
    session_id=sessionID["session_id"],
        input={
        'message_type': 'text',
        'text': input_data
        }
    ).get_result()
    choice = False
    zipcode = ""
    text = response["output"]["generic"][0]["text"]
    intent = response["output"]["intents"][0]["intent"] if response["output"]["intents"] else ""
#     print(json.dumps(response, indent=2))
    choices = []
    if len(response["output"]['generic']) > 1 and "options" in response["output"]['generic'][1]:
        choice = True
        for opt in response["output"]['generic'][1]["options"]:
            choices.append(opt["label"])

    if intent in entity_choices:
        preference["place"] = intent
    elif intent in feature_choices:
        if intent == "Price":
            print(input_data)
            preference["price"] = "high" if 'Expensive' in input_data else "low"
        elif intent == "rating":
            preference['rating'] = "low" if input_data == "I do not care much about rating" else "high"
        elif intent == 'Room_type':
            preference['bed'] = input_data
    if response["output"]['entities']:
        if response["output"]['entities'][0]['entity'] == 'zipcode':
            zipcode = response["output"]['entities'][0]['value']
            preference['zipcode'] = input_data
    if not choice:
        return text, None, preference
    else:
        return text, choices, preference
    
def main():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    # Use no used localhost and port.
    sock.bind(('127.0.0.1', 9996))
    sock.listen(5)
    url = current_path + "/webpage/main.html"
    webbrowser.open(url, new = 2)
    while True:
        try:
            connection, address = sock.accept()
            socketProgramming(connection).start()
        except:
            time.sleep(1)

/Users/jessieyee/opt/anaconda3/envs/final_bigdata/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
if __name__ == "__main__":
    main()

{'zipcode': '10003', 'place': 'restaurant'}
{'zipcode': '10003', 'place': 'restaurant'}
{'restaurant': {'price': 'no_require', 'rating': 'no_require', 'safety': 'high'}, 'play': {'rating': 'high', 'safety': 'high'}}


Exception in thread Thread-8:
Traceback (most recent call last):
  File "/Users/jessieyee/opt/anaconda3/envs/final_bigdata/lib/python3.8/site-packages/pandas_gbq/gbq.py", line 507, in _download_results
    query_job.result()
  File "/Users/jessieyee/opt/anaconda3/envs/final_bigdata/lib/python3.8/site-packages/google/cloud/bigquery/job/query.py", line 1447, in result
    do_get_result()
  File "/Users/jessieyee/opt/anaconda3/envs/final_bigdata/lib/python3.8/site-packages/google/api_core/retry.py", line 283, in retry_wrapped_func
    return retry_target(
  File "/Users/jessieyee/opt/anaconda3/envs/final_bigdata/lib/python3.8/site-packages/google/api_core/retry.py", line 190, in retry_target
    return target()
  File "/Users/jessieyee/opt/anaconda3/envs/final_bigdata/lib/python3.8/site-packages/google/cloud/bigquery/job/query.py", line 1437, in do_get_result
    super(QueryJob, self).result(retry=retry, timeout=timeout)
  File "/Users/jessieyee/opt/anaconda3/envs/final_bigdata/lib/python